In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import gc
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import os
tf.executing_eagerly()

from tensorflow import keras

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.63.0.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.63.0.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
df=pd.read_csv(r'/content/drive/MyDrive/Devopedia/all_full_5.csv')
df_author_diagnostics=pd.read_csv(r'/content/drive/MyDrive/Devopedia/Author_diagnostics_full_5.csv')
df.drop(['Title','Author','YoPublishing','Title_encoding','YoP_encoding'],axis=1,inplace=True)

In [ ]:
print("Raw Dataset")
print(df.shape)
print("No of 0s: ",len(df[df.Author_encoding==0]))
print("No of 1s: ",len(df[df.Author_encoding==1]))
print("0:1 ratio:",len(df[df.Author_encoding==0])/len(df[df.Author_encoding==1]))

Raw Dataset
(2194434, 5)
No of 0s:  2182944
No of 1s:  11490
0:1 ratio: 189.9864229765013


In [ ]:
drop_files=[]

for i in tqdm(df_author_diagnostics.iterrows()):
    if i[1].Distinct_authors_count==0:
        drop_files.append(i[1].Fname)

6761it [00:00, 10565.15it/s]


In [ ]:
df= df[~df.fname.isin(drop_files)]

In [ ]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [ ]:
print("After dropping files")
print(df.shape)
print(len(df[df.Author_encoding==0])/len(df[df.Author_encoding==1]))

After dropping files
(1634264, 5)
141.23359442993907


In [ ]:
tag_special_char_check=re.compile('[^a-zA-Z]')

one_hot=pd.get_dummies(df.Tag)

In [ ]:
one_hot.shape

(1634264, 301)

In [ ]:
allowed_tags=[]

for i in tqdm(one_hot):
  if tag_special_char_check.search(i)==None:
    allowed_tags.append(i)

del one_hot
gc.collect()


  0%|          | 301/1634264 [00:00<00:10, 151286.46it/s]


22

In [ ]:
df=df[df.Tag.isin(allowed_tags)]

In [ ]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [ ]:
df

,fname,Author_encoding,SerialNo,Tag,Text
0,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,1,title,Full Page Reload
1,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,2,header,Join IEEE | IEEE.org | IEEE Xplore Digital Lib...
2,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,3,ul,Join IEEE | IEEE.org | IEEE Xplore Digital Lib...
3,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,4,li,Join IEEE
4,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,5,li,IEEE.org
...,...,...,...,...,...
1564177,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,1,2,head,LoRaTools
1564178,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,3,body,Toggle navigation LoRaTools {{item.name}} var ...
1564179,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,4,div,Toggle navigation LoRaTools {{item.name}}
1564180,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,5,div,Toggle navigation LoRaTools


In [ ]:
print("After removing tags with special characters")
print(df.shape)
print(len(df[df.Author_encoding==0])/len(df[df.Author_encoding==1]))

After removing tags with special characters
(1564182, 5)
141.2112919356305


In [ ]:
df

,fname,Author_encoding,SerialNo,Tag,Text
0,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,1,title,Full Page Reload
1,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,2,header,Join IEEE | IEEE.org | IEEE Xplore Digital Lib...
2,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,3,ul,Join IEEE | IEEE.org | IEEE Xplore Digital Lib...
3,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,4,li,Join IEEE
4,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,5,li,IEEE.org
...,...,...,...,...,...
1564177,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,1,2,head,LoRaTools
1564178,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,3,body,Toggle navigation LoRaTools {{item.name}} var ...
1564179,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,4,div,Toggle navigation LoRaTools {{item.name}}
1564180,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,5,div,Toggle navigation LoRaTools


In [ ]:
from sklearn.preprocessing import OneHotEncoder

OHE=OneHotEncoder(handle_unknown='ignore',sparse=False)

In [ ]:
OHE.fit(np.array(df['Tag']).reshape(-1,1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=False)

In [ ]:
df_author_diagnostics_subset= df_author_diagnostics.sample(n=584)

In [ ]:
df_author_diagnostics_subset

,Fname,Distinct_authors_count,Authors
3873,547c044c791a4496fb074b3e0131401449c53d8a8d02b7...,1,"Weaver, Rick"
1315,096d22c3822ea17523e7e003e38029ebf112b20927d759...,0,NewGenApps
2781,f23762c87425b1b67cb4181096c2177133fd1b9bd2b58a...,1,"Coyier, Chris"
5039,6403f2ebed03dd3d470008e28fc398c6743c4e55851fb6...,1,Cipher
6109,2dde85d33ee9ea9cd654885dbb6429527fb1f2b9d33a37...,3,"Zhao, Shubin, and Ralph Grishman"
...,...,...,...
3288,c1567aab29e70f7ee967496c892bca0ad7b74aabcbee11...,1,"Mhetre, Abhijit"
1664,75016659f439a357c5e7290d6432bad0ee427ed1242a16...,0,DataFlair
2259,e442e5245fd753c60600b79a8f81cebaaaf338dfdc5bfb...,1,"Florance, Ken"
763,bfed48be63141f8f25e89f9c0dc435031b981de164d52e...,0,"Prokop, Andrew"


In [ ]:
test_fnames=list(df_author_diagnostics_subset.Fname)

In [ ]:
df_test= df[df.fname.isin(test_fnames)]
df_test.reset_index(inplace=True)
df_test.drop('index',axis=1,inplace=True)

df_train=df[~df.fname.isin(test_fnames)]
df_train.reset_index(inplace=True)
df_train.drop('index',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
print("Train Dataset")
print(df_train.shape)
print("No of 0s: ",len(df_train[df_train.Author_encoding==0]))
print("No of 1s: ",len(df_train[df_train.Author_encoding==1]))
print("0:1 ratio:",len(df_train[df_train.Author_encoding==0])/len(df_train[df_train.Author_encoding==1]))
print("-"*50)
print("Test Dataset")
print(df_test.shape)
print("No of 0s: ",len(df_test[df_test.Author_encoding==0]))
print("No of 1s: ",len(df_test[df_test.Author_encoding==1]))
print("0:1 ratio:",len(df_test[df_test.Author_encoding==0])/len(df_test[df_test.Author_encoding==1]))

Train Dataset
(1432760, 5)
No of 0s:  1422726
No of 1s:  10034
0:1 ratio: 141.7905122583217
--------------------------------------------------
Test Dataset
(131422, 5)
No of 0s:  130457
No of 1s:  965
0:1 ratio: 135.18860103626943


In [ ]:
df_test

,fname,Author_encoding,SerialNo,Tag,Text
0,001a52277fbca48c983f0b66e3af702facd66153a4fcb2...,0,1,title,Beam Mobile Gaming Example
1,001a52277fbca48c983f0b66e3af702facd66153a4fcb2...,0,2,div,Get Started Documentation Toggle navigation
2,001a52277fbca48c983f0b66e3af702facd66153a4fcb2...,0,3,a,Get Started
3,001a52277fbca48c983f0b66e3af702facd66153a4fcb2...,0,4,a,Documentation
4,001a52277fbca48c983f0b66e3af702facd66153a4fcb2...,0,5,span,Toggle navigation
...,...,...,...,...,...
131417,ff29095c6ac32ac293c15aed92fa7542099b20eb9edac4...,0,146,li,Nondiscrimination Policy
131418,ff29095c6ac32ac293c15aed92fa7542099b20eb9edac4...,0,147,li,IEEE Privacy Policy
131419,ff29095c6ac32ac293c15aed92fa7542099b20eb9edac4...,1,148,li,IEEE SA Copyright Policy
131420,ff29095c6ac32ac293c15aed92fa7542099b20eb9edac4...,0,149,a,IEEE Terms and Conditions


In [ ]:
df_train

,fname,Author_encoding,SerialNo,Tag,Text
0,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,1,title,Full Page Reload
1,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,2,header,Join IEEE | IEEE.org | IEEE Xplore Digital Lib...
2,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,3,ul,Join IEEE | IEEE.org | IEEE Xplore Digital Lib...
3,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,4,li,Join IEEE
4,00031cefe68278406ddac7764d46275a56a13558c3bfee...,0,5,li,IEEE.org
...,...,...,...,...,...
1432755,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,1,2,head,LoRaTools
1432756,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,3,body,Toggle navigation LoRaTools {{item.name}} var ...
1432757,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,4,div,Toggle navigation LoRaTools {{item.name}}
1432758,fffe5eb5158dc707f949370abe44dd3d6ce77f63010156...,0,5,div,Toggle navigation LoRaTools


In [ ]:
caps=[]
first_token_upper=[]
comma_percent=[]
no_of_tokens=[]


for i in tqdm(df_train.iterrows()):
    text=str(i[1].Text)
    no_toks = len(re.findall(r'\w+', text))

    x=len(re.findall(r'[A-Z]',text))
    caps.append(x)

    no_of_tokens.append(no_toks)
    
    try:
      comma_percent.append(text.count(',')/no_toks)
    except:
      comma_percent.append(0)
    
    if text.split()[0].isupper():
        first_token_upper.append(1)
    else:
        first_token_upper.append(0)
    

1432760it [02:37, 9098.79it/s]


In [ ]:
df_train['caps_count']=np.array(caps)
df_train['first_token_upper']=np.array(first_token_upper)
df_train['comma_percent']=np.array(comma_percent)
df_train['No_of_tokens']=np.array(no_of_tokens)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
caps=[]
first_token_upper=[]
comma_percent=[]
no_of_tokens=[]


for i in tqdm(df_test.iterrows()):
    text=str(i[1].Text)
    no_toks = len(re.findall(r'\w+', text))

    x=len(re.findall(r'[A-Z]',text))
    caps.append(x)

    no_of_tokens.append(no_toks)
    
    try:
      comma_percent.append(text.count(',')/no_toks)
    except:
      comma_percent.append(0)
    
    if text.split()[0].isupper():
        first_token_upper.append(1)
    else:
        first_token_upper.append(0)
    

131422it [00:14, 9179.61it/s]


In [ ]:
len(no_of_tokens)

131422

In [ ]:
df_test['caps_count']=np.array(caps)
df_test['first_token_upper']=np.array(first_token_upper)
df_test['comma_percent']=np.array(comma_percent)
df_test['No_of_tokens']=np.array(no_of_tokens)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432760 entries, 0 to 1432759
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   fname              1432760 non-null  object 
 1   Author_encoding    1432760 non-null  int64  
 2   SerialNo           1432760 non-null  int64  
 3   Tag                1432760 non-null  object 
 4   Text               1432594 non-null  object 
 5   caps_count         1432760 non-null  int64  
 6   first_token_upper  1432760 non-null  int64  
 7   comma_percent      1432760 non-null  float64
 8   No_of_tokens       1432760 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 98.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131422 entries, 0 to 131421
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fname              131422 non-null  object 
 1   Author_e

In [ ]:
print("Train Dataset")
print(df_train.shape)
print("No of 0s: ",len(df_train[df_train.Author_encoding==0]))
print("No of 1s: ",len(df_train[df_train.Author_encoding==1]))
print("0:1 ratio:",len(df_train[df_train.Author_encoding==0])/len(df_train[df_train.Author_encoding==1]))
print("-"*50)
print("Test Dataset")
print(df_test.shape)
print("No of 0s: ",len(df_test[df_test.Author_encoding==0]))
print("No of 1s: ",len(df_test[df_test.Author_encoding==1]))
print("0:1 ratio:",len(df_test[df_test.Author_encoding==0])/len(df_test[df_test.Author_encoding==1]))

Train Dataset
(1432760, 9)
No of 0s:  1422726
No of 1s:  10034
0:1 ratio: 141.7905122583217
--------------------------------------------------
Test Dataset
(131422, 9)
No of 0s:  130457
No of 1s:  965
0:1 ratio: 135.18860103626943


In [ ]:
OH_train_cols=pd.DataFrame(OHE.transform(np.array(df_train['Tag']).reshape(-1,1)))
OH_test_cols=pd.DataFrame(OHE.transform(np.array(df_test['Tag']).reshape(-1,1)))

In [ ]:
print(OH_train_cols.shape)
print(OH_test_cols.shape)

(1432760, 146)
(131422, 146)


In [ ]:
df_train=df_train.join(OH_train_cols)
del OH_train_cols
gc.collect()

200

In [ ]:
df_test=df_test.join(OH_test_cols)
del OH_test_cols
gc.collect()

50

In [ ]:
df_train.drop(['fname','Tag','Text'],axis=1,inplace=True)
df_test.drop(['fname','Tag','Text'],axis=1,inplace=True)

In [ ]:
print(df_train.shape)
print(df_test.shape)

(1432760, 152)
(131422, 152)


In [ ]:
df_train=df_train[df_train.SerialNo<1009]

df_train0=df_train[df_train.Author_encoding==0]
df_train1=df_train[df_train.Author_encoding==1]

df_test=df_test[df_test.SerialNo<1009]

df_test0=df_test[df_test.Author_encoding==0]
df_test1=df_test[df_test.Author_encoding==1]

In [ ]:
print(df_train1.shape)
print(df_train0.shape)

(9802, 152)
(1171989, 152)


In [ ]:
df_train1=pd.concat([df_train1]*3, ignore_index=True)


In [ ]:
print(df_train1.shape)
print(df_train0.shape)

(29406, 152)
(1171989, 152)


In [ ]:
print(df_train.shape)
print(df_test.shape)

print(f'Train 0:1 proportion:       {len(df_train0)/len(df_train1)}')
print(f'Test 0:1 proportion:       {len(df_test0)/len(df_test1)}')

(1181791, 152)
(117572, 152)
Train 0:1 proportion:       39.855437665782496
Test 0:1 proportion:       125.69396551724138


In [ ]:
df_train.reset_index(inplace=True)
df_train.drop('index',axis=1,inplace=True)

df_test.reset_index(inplace=True)
df_test.drop('index',axis=1,inplace=True)

In [ ]:
negs=len(df_train0)
pos=len(df_train1)
print ('Negatives:',negs )
print ('Positives:', pos)
total=negs+pos

Negatives: 1171989
Positives: 29406


In [ ]:
weight_for_0 = (1 / negs) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

In [ ]:
class_weight

{0: 0.5125453395893648, 1: 20.427718832891244}

In [ ]:
# del df_train
# gc.collect()
# del df_test
# gc.collect()

In [ ]:
target1 = df_train1.pop('Author_encoding')
train_dataset1 = tf.data.Dataset.from_tensor_slices((df_train1.values, target1.values))

del df_train1
gc.collect()

target0 = df_train0.pop('Author_encoding')
train_dataset0 = tf.data.Dataset.from_tensor_slices((df_train0.values, target0.values))

del df_train0
gc.collect()

0

In [ ]:
target1 = df_test1.pop('Author_encoding')
test_dataset1 = tf.data.Dataset.from_tensor_slices((df_test1.values, target1.values))

del df_test1
gc.collect()

target0 = df_test0.pop('Author_encoding')
test_dataset0 = tf.data.Dataset.from_tensor_slices((df_test0.values, target0.values))

del df_test0
gc.collect()

0

In [ ]:
train_dataset_sampled=tf.data.experimental.sample_from_datasets([train_dataset0, train_dataset1], weights=[0.75, 0.25])
del train_dataset1
del train_dataset0
gc.collect()

test_dataset_sampled=tf.data.experimental.sample_from_datasets([test_dataset0, test_dataset1], weights=[0.75, 0.25])
del test_dataset1
del test_dataset0
gc.collect()

0

In [ ]:
train_dataset_sampled=train_dataset_sampled.batch(1024)
test_dataset_sampled=test_dataset_sampled.batch(1024)

In [ ]:
for feat, targ in train_dataset_sampled.take(2):
  print ('{}'.format(targ),end=" ")

[1 0 0 ... 0 0 1] [0 1 0 ... 1 0 0] 

In [ ]:
train_dataset_sampled

<BatchDataset shapes: ((None, 151), (None,)), types: (tf.float64, tf.int64)>

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
inputs = keras.Input(shape=(151))
x = layers.Dense(256, activation="relu", name="first_layer")(inputs)
x = layers.Dense(128, activation="relu", name="second_layer")(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs,name="Basic_ANN")

model.summary()

Model: "Basic_ANN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 151)]             0         
_________________________________________________________________
first_layer (Dense)          (None, 256)               38912     
_________________________________________________________________
second_layer (Dense)         (None, 128)               32896     
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 72,066
Trainable params: 72,066
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)

In [ ]:
history=model.fit(train_dataset_sampled, epochs=20, verbose=1,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Epoch 1/20
1174/1174 [==============================] - 16s 12ms/step - loss: 0.3034 - accuracy: 0.9218
Epoch 2/20
1174/1174 [==============================] - 15s 12ms/step - loss: 4.5841 - accuracy: 0.9201
Epoch 3/20
1174/1174 [==============================] - 15s 12ms/step - loss: 3.5413 - accuracy: 0.9240
Epoch 4/20
1174/1174 [==============================] - 15s 12ms/step - loss: 4.7795 - accuracy: 0.9246
Epoch 5/20
1174/1174 [==============================] - 15s 12ms/step - loss: 1.9220 - accuracy: 0.9231
Epoch 6/20
1174/1174 [==============================] - 15s 12ms/step - loss: 2.2062 - accuracy: 0.9169
Epoch 7/20
1174/1174 [==============================] - 15s 12ms/step - loss: 1.2203 - accuracy: 0.9243
Epoch 8/20
1174/1174 [==============================] - 15s 12ms/step - loss: 3.1784 - accuracy: 0.9155
Epoch 9/20
1174/1174 [==============================] - 15s 12ms/step - loss: 2.1771 - accuracy: 0.9181
Epoch 10/20
1174/1174 [==============================] - 15s 12m

In [ ]:
model.evaluate(test_dataset_sampled)

115/115 [==============================] - 2s 18ms/step - loss: 0.1031 - accuracy: 0.9909


[0.10307913273572922, 0.9908992052078247]

In [ ]:
y_pred = model.predict(test_dataset_sampled)

predicted_categories = tf.argmax(y_pred, axis=1)

true_categories = tf.concat([y for x, y in test_dataset_sampled], axis=0)


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(predicted_categories, true_categories)

array([[116456,    918],
       [   188,     10]])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(predicted_categories, true_categories))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00    117374
           1       0.01      0.05      0.02       198

    accuracy                           0.99    117572
   macro avg       0.50      0.52      0.51    117572
weighted avg       1.00      0.99      0.99    117572

